# Producer Consumer Basic

## This is wrong

In [ ]:
import threading
from threading import Condition, Lock
import time
import random
queue = []
lock = Lock()

def produce():
    nums = range(5) #Will create the list [0, 1, 2, 3, 4]
    global queue
    while True:
        num = random.choice(nums) #choose a random number from nums
        lock.acquire()
        queue.append(num)
        print("Produced", num)
        lock.release()
        time.sleep(10)

def consume():
    global queue
    while True:
        lock.acquire()
        if not queue:
            print ("Nothing in queue, Wait")
        else:          
            num = queue.pop(0)
            print("Consumed", num)
        lock.release()
        time.sleep(1)
        
p=threading.Thread(target=produce)
c=threading.Thread(target=consume)
p.start()
c.start()
p.join()
c.join()
time.sleep(1)
print("Finished")

## This is better but with no buffer size

In [ ]:
import threading
from threading import Condition, Lock
import time
import random
queue = []
condition = Condition()

def produce():
    nums = range(5) #Will create the list [0, 1, 2, 3, 4]
    global queue
    while True:
        num = random.choice(nums) #choose a random number from nums
        condition.acquire()
        queue.append(num)
        print("Produced", num, "queue size", len(queue))
        condition.notify()
        condition.release()
        time.sleep(10)

def consume():
    global queue
    while True:
        condition.acquire()
        if not queue:
            print ("Nothing in queue, Wait")
            condition.wait()
        num = queue.pop(0)
        print("Consumed", num)
        condition.release()
        time.sleep(1)
        
p=threading.Thread(target=produce)
c=threading.Thread(target=consume)
p.start()
c.start()
p.join()
c.join()
time.sleep(1)
print("Finished")

Produced 4 queue size 1
Consumed 4
Nothing in queue, Wait
Produced 1 queue size 1
Consumed 1
Nothing in queue, Wait
Produced 3 queue size 1
Consumed 3
Nothing in queue, Wait
Produced 0 queue size 1
Consumed 0
Nothing in queue, Wait
Produced 2 queue size 1
Consumed 2


# Buffer Size

In [ ]:
import threading
from threading import Condition
import time
import random
queue = []
condition = Condition()

def produce():
    nums = range(5) #Will create the list [0, 1, 2, 3, 4]
    global queue
    MAX_NUM = 10
    
    while True:
        num = random.choice(nums) #choose a random number from nums
        condition.acquire()
        if len(queue) == MAX_NUM:
            print ("Full buffer")
            condition.wait()
            print ("Buffer ready, Consumer notified the producer")       
        queue.append(num)
        print("Produced", num, "queue size", len(queue))
        condition.notify()
        condition.release()
        time.sleep(1)

def consume():
    global queue
    while True:
        condition.acquire()
        if not queue:
            print ("Nothing in queue, Wait")
            condition.wait()
        num = queue.pop(0)
        print("Consumed", num)
        condition.notify()
        condition.release()
        time.sleep(10)
        
p=threading.Thread(target=produce)
c=threading.Thread(target=consume)
p.start()
c.start()
p.join()
c.join()
time.sleep(1)
print("Finished")

Produced 0 queue size 1
Consumed 0
Produced 4 queue size 1
Produced 1 queue size 2
Produced 4 queue size 3
Produced 4 queue size 4
Produced 3 queue size 5
Produced 1 queue size 6
Produced 0 queue size 7
Produced 2 queue size 8
Produced 4 queue size 9
Consumed 4
Produced 3 queue size 9
Produced 0 queue size 10
Full buffer
Consumed 1
Buffer ready, Consumer notified the producer
Produced 2 queue size 10
Full buffer
Consumed 4
Buffer ready, Consumer notified the producer
Produced 0 queue size 10
Full buffer
